In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
#Load the data
data = pd.read_csv(r'C:\Users\snehal.sankanna\AppData\Local\anaconda3\Churn_Mdelling\Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [26]:
data.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited',
       'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [3]:
#Drop irrelevent columns
data.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [4]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [5]:
#For encoding of Geography column onehot encoder is used because if by using labelencoder model can consider higher labelled value more imp
onehotencoder_geo = OneHotEncoder()
encoded_geo = onehotencoder_geo.fit_transform(data[['Geography']])

In [6]:
encoded_geo_df = pd.DataFrame(
    encoded_geo.toarray(),
    columns=onehotencoder_geo.get_feature_names_out(['Geography'])
)

In [7]:
data = pd.concat([data.reset_index(drop=True), encoded_geo_df],axis=1)
data.drop(columns=['Geography'], axis=1, inplace=True)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
#save the encoders as pickle
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehotencoder_geo.pkl', 'wb') as file:
    pickle.dump(onehotencoder_geo, file)

In [9]:
#Divide data into dependent and independent features
X = data.drop(['Exited'], axis=1)
y = data['Exited']

In [10]:
#Splitting train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
#Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

ANN Implementation

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
#used to store the training logs in folder and visualize them so that user can understand them

In [14]:
model =Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
X_train.shape[1]

12

In [17]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
#If we want to change the default learning rate of optimizers
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()
#model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [19]:
#log_dir = "log/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "log/fit/"

tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [20]:
#Early stopping callback
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [21]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=[tensorflow_callback, early_stopping_callback])

Epoch 1/100


250/250 [==============================] - 4s 12ms/step - loss: 0.4516 - accuracy: 0.8058 - val_loss: 0.3900 - val_accuracy: 0.8405
Epoch 2/100
250/250 [==============================] - 2s 8ms/step - loss: 0.3846 - accuracy: 0.8415 - val_loss: 0.3596 - val_accuracy: 0.8525
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3564 - accuracy: 0.8555 - val_loss: 0.3480 - val_accuracy: 0.8570
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3438 - accuracy: 0.8586 - val_loss: 0.3458 - val_accuracy: 0.8545
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3369 - accuracy: 0.8616 - val_loss: 0.3451 - val_accuracy: 0.8625
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3335 - accuracy: 0.8624 - val_loss: 0.3410 - val_accuracy: 0.8555
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3302 - accuracy: 0.8639 - val_loss: 0.3369 - val_accuracy: 0.8

In [22]:
model.save('model.h5')

c:\Users\snehal.sankanna\AppData\Local\anaconda3\envs\myenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
#Load tensorboard extension
%load_ext tensorboard

In [24]:
log_dir

'log/fit/'

In [25]:
%tensorboard --logdir log/fit/


Reusing TensorBoard on port 6007 (pid 6260), started 0:03:35 ago. (Use '!kill 6260' to kill it.)